In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from common import utils, vocabulary, tf_embed_viz, treeviz
from common import patched_numpy_io

from nltk.tokenize import sent_tokenize


from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from time import time

In [2]:
# extracting possibly conservative sentences
reddit_the_donald = pd.read_csv("~/political_bias/political_bias/reddit_data_queries/the_donald_posts.csv")

In [3]:
reddit_the_donald.columns = ['sentence']

In [4]:
r = 'com/2017/12/13/breaking-senator is a shame'

r_split = r.split(" ")
print(len(r_split))
print(r_split)
sent1 = []
for s in r_split:
    if not(re.match(r'([a-z_\-\s0-9\.]+)+/',s)):
        sent1.append(s)
print(" ".join(sent1))        

4
['com/2017/12/13/breaking-senator', 'is', 'a', 'shame']
is a shame


In [5]:
sent = "[*Muhammad was married to thirteen women, /com/78/89/gama including eleven at one time. He relegated them to either consecutive days or (according to some accounts) all in one night. He had sex with a 9-year-old girl and married his adopted son's wife (after arranging a quick divorce). On top of that, Muhammad had a multitude of slave girls and concubines with whom he had sex - sometimes on the very days in which they watched their husbands and fathers die at the hands of his army.*](https://www.thereligionofpeace.com/pages/quran/muhammads-sex-life.aspx)"

def identify_links(sent):
    if isinstance(sent,str):
        r_split = sent.split(" ")
        sent1 = []
        for s in r_split:
            if not(re.match(r'/?([a-z_\-\s0-9\.]+)+/',s)):
                sent1.append(s)
        return " ".join(sent1)
identify_links(sent)

"[*Muhammad was married to thirteen women, including eleven at one time. He relegated them to either consecutive days or (according to some accounts) all in one night. He had sex with a 9-year-old girl and married his adopted son's wife (after arranging a quick divorce). On top of that, Muhammad had a multitude of slave girls and concubines with whom he had sex - sometimes on the very days in which they watched their husbands and fathers die at the hands of his army.*](https://www.thereligionofpeace.com/pages/quran/muhammads-sex-life.aspx)"

In [6]:
print(reddit_the_donald.loc[1]['sentence'])
print(identify_links(reddit_the_donald.loc[1]['sentence']))

[*Muhammad was married to thirteen women, including eleven at one time. He relegated them to either consecutive days or (according to some accounts) all in one night. He had sex with a 9-year-old girl and married his adopted son's wife (after arranging a quick divorce). On top of that, Muhammad had a multitude of slave girls and concubines with whom he had sex - sometimes on the very days in which they watched their husbands and fathers die at the hands of his army.*](https://www.thereligionofpeace.com/pages/quran/muhammads-sex-life.aspx)
[*Muhammad was married to thirteen women, including eleven at one time. He relegated them to either consecutive days or (according to some accounts) all in one night. He had sex with a 9-year-old girl and married his adopted son's wife (after arranging a quick divorce). On top of that, Muhammad had a multitude of slave girls and concubines with whom he had sex - sometimes on the very days in which they watched their husbands and fathers die at the han

In [7]:
import chardet

X_windows1252 = []

for i in range(reddit_the_donald['sentence'].count()):
    if isinstance(reddit_the_donald.loc[i]['sentence'],str):
        my_byte = str.encode(reddit_the_donald.loc[i]['sentence'])
        if chardet.detect(my_byte)['encoding'] == 'Windows-1252':
            X_windows1252.append(identify_links(reddit_the_donald.loc[i]['sentence']))
            

In [8]:
len(X_windows1252)

93

In [9]:
sent = 'http://mon.com/2017/12/13/breaking-senator is a shame'

def identify_url(sent):
    if isinstance(sent,str):
        r_split = sent.split(" ")
        sent1 = []
        for s in r_split:
            if not(re.match('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',s)):
                sent1.append(s)
        return " ".join(sent1)
identify_url(sent)   

'is a shame'

In [10]:
sent = "[*http://detcon.com/boul/cup is fine"

def remove_spsymb(sent):
    if isinstance(sent,str):
        r_split = sent.split(' ')

        sent1 = []
        for s in r_split:
            sent1.append(re.sub(r'[()[\]{}*+=^]+',' ',s))
        return " ".join(sent1)
remove_spsymb(sent)

' http://detcon.com/boul/cup is fine'

In [11]:
X_windows1252 = pd.DataFrame(data=X_windows1252,columns = ['sentence'])

In [12]:
X_windows1252.sentence = X_windows1252.sentence.apply(lambda x: remove_spsymb(x))

In [13]:
X_windows1252.sentence = X_windows1252.sentence.apply(lambda x: identify_url(x))

In [14]:
X_windows1252.sentence = X_windows1252.sentence.apply(lambda x: identify_links(x))

In [15]:
X_windows1252.sentence[92]

'---  Intro and Motivation   Given what we know now in late Dec, 2017, the continuance of the obvious witch hunt of Mueller\'s "Special Council" is now causing harm to the American people, because it\'s sending a signal to our culture and to our legal system that  "presumption of innocence"  doesn\'t matter--it is false.  Because if a group can do this to the highest position in america--the President of the United States--they can certainly do this same thing to you:  lawfare .  Dragging out a legal matter, well beyond its intended scope and time frame, causing grievous harm to taxpayers who have to shoulder the burden of the same, while also insulting and accusing not just the lawfully-elected President but all of those who voted for him.  In a way, this accusation of election meddling with no evidence to support it--with plenty of evidence to disprove the same--is an attempt not only to reverse a disfavorable political decision by the oppositional team,  ...the continuance of the sp

In [16]:
import string

def clean_numbers(sentence):
    translator = str.maketrans(dict.fromkeys(string.digits))
    
    return sentence.translate(translator)

In [17]:
X_windows1252.sentence = X_windows1252.sentence.apply(lambda x: clean_numbers(x))

In [18]:
r = string.punctuation
r

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
d = {r[0]:None,r[1]:None, r[2]:None,r[3]:None,r[4]:None,r[5]:" ",r[6]: None,r[7]:None,r[10]:" ",r[11]:None,r[12]:" ",r[13]:None,r[14]:" ",
    r[15]:" ",r[16]:None,r[17]:None,r[19]:None,r[20]:None,r[21]:" ",r[26]:" ",r[27]:None,r[29]:" ",r[31]:" "}

In [20]:
r[5]

'&'

In [21]:
def clean_punc(sentence):
    translator = str.maketrans(d)
    
    return sentence.translate(translator)

In [22]:
X_windows1252.sentence = X_windows1252.sentence.apply(lambda x: clean_punc(x))

In [23]:
reddit_bernie = pd.read_csv('~/political_bias/political_bias/reddit_data_queries/reddit_sanders.csv')

In [24]:
reddit_bernie.columns = ['sentence']

In [25]:
X1_windows1252 = []

for i in range(reddit_bernie['sentence'].count()):
    if isinstance(reddit_bernie.loc[i]['sentence'],str):
        my_byte = str.encode(reddit_bernie.loc[i]['sentence'])
        if chardet.detect(my_byte)['encoding'] == 'Windows-1252':
            X1_windows1252.append(reddit_bernie.loc[i]['sentence'])

In [26]:
len(X1_windows1252)

206

In [27]:
X1_windows1252 = pd.DataFrame(data=X1_windows1252,columns = ['sentence'])

In [28]:
X1_windows1252.sentence = X1_windows1252.sentence.apply(lambda x: remove_spsymb(x))

In [29]:
X1_windows1252.sentence = X1_windows1252.sentence.apply(lambda x: identify_url(x))

In [30]:
X1_windows1252.sentence = X1_windows1252.sentence.apply(lambda x: clean_punc(x))

In [31]:
X1_windows1252.sentence[100]

'I have a challenge for those in the Loveland   Berthoud area in Colorado  We are all working to get Bernie nominated next year that will be followed by the real work to get him elected in November 2016  Lets do and an experiment in a local election this year 2015 Loveland Colorado has a municipal election this year  Nov 3  for Thompson School Board and Loveland city council Two years ago the school board was taken over by the by those supported by Americans for Prosperity and other grups Now the Thompson School Board looks like Douglas County with a dysfunctional board Our city council is fighting a similar battle along with the school board this year So Im suggesting that we take some of our energy and mobilize our forces for the next 3 weeks and see what we can do get our supporters to vote in this election Ballots will be mailed after Oct 12 this year Then see what worked and what didn’t work This will give us a model to use on Bernies campaign with lessons learned  Something to co

In [32]:
X_windows1252.to_csv('reddit_the_donald.csv')

In [33]:
X1_windows1252.to_csv('reddit_bernie.csv')

In [34]:
sentences = []
for i in range(X_windows1252.sentence.count()):
    sentences.append(X_windows1252.sentence[i])

In [35]:
# now we try clustering
vectorizer = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=False,analyzer='word')

In [36]:
X = vectorizer.fit_transform(sentences)

In [37]:
X

<93x2673 sparse matrix of type '<class 'numpy.float64'>'
	with 3753 stored elements in Compressed Sparse Row format>

In [38]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

In [39]:
model1 = model.fit_transform(X)

In [40]:
type(model1)

numpy.ndarray

In [41]:
model1.shape

(93, 2)

In [42]:
len(vectorizer.get_feature_names())

2673

In [43]:
order_descent= model.cluster_centers_.argsort()[:, ::-1]

In [44]:
order_descent

array([[ 723,  514,  725, ..., 1038, 1041, 2672],
       [ 376,  743, 1530, ..., 1496, 1497,    0]])

In [45]:
terms = vectorizer.get_feature_names()

In [46]:
order_descent.shape

(2, 2673)

In [48]:
len(terms)

2673

In [51]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent[i, :55]:
        print(' %s' % terms[ind]),

Cluster 0:
 That
 Now
 The
 year
 like
 He
 seen
 just
 confused
 right
 time
 don
 want
 really
 know
 Why
 Just
 Dingo
 GEOTUS
 ve
 employees
 life
 Lol
 post
 come
 What
 Can
 They
 follow
 hilarious
 And
 knew
 fun
 Christmas
 old
 taking
 election
 Sen
 Still
 haven
 cut
 American
 Alabama
 twitter
 happy
 In
 When
 guy
 make
 If
 tax
 As
 good
 apply
 years
Cluster 1:
 It
 Trump
 great
 right
 people
 entertainer
 This
 DEADFEDDEEDB
 coverage
 Try
 hilarious
 think
 feels
 All
 make
 Moore
 movie
 wasn
 referendum
 vote
 money
 inbred
 REAL
 Intelligence
 American
 times
 blah
 negative
 They
 doesn
 week
 world
 simultaneously
 Jr
 terrifying
 shadow
 literally
 ban
 laws
 Clinton
 trump
 MAGA
 article
 long
 stopped
 matter
 smug
 globalist
 ass
 views
 There
 wait
 broken
 And
 effect


In [58]:
sentences = []
for i in range(X1_windows1252.sentence.count()):
    sentences.append(X1_windows1252.sentence[i])

In [59]:
# now we try clustering
vectorizer1 = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=False,analyzer='word')

In [60]:
X1 = vectorizer1.fit_transform(sentences)

In [61]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

In [62]:
model2 = model.fit_transform(X1)

In [63]:
order_descent1= model.cluster_centers_.argsort()[:, ::-1]

In [64]:
terms1 = vectorizer1.get_feature_names()

In [65]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent1[i, :55]:
        print(' %s' % terms1[ind]),

Cluster 0:
 Bernie
 people
 Sanders
 The
 like
 gt
 We
 know
 think
 This
 want
 make
 If
 just
 vote
 campaign
 issues
 support
 time
 voters
 Hillary
 dont
 state
 candidate
 It
 right
 work
 money
 Bernies
 help
 need
 important
 Im
 political
 way
 women
 candidates
 believe
 And
 issue
 really
 He
 good
 say
 information
 election
 does
 event
 lot
 said
 primary
 free
 actually
 party
 better
Cluster 1:
 Sanders
 Clinton
 Bernie
 percent
 The
 Hillary
 just
 Mentions
 poll
 campaign
 Super
 Democratic
 article
 Bernies
 Blackout
 think
 11
 Senator
 Times
 PACK
 We
 Day
 Change
 media
 10
 support
 animation
 money
 good
 In
 like
 candidate
 And
 new
 30
 feedback
 Mr
 If
 people
 help
 presidential
 getting
 Facebook
 NH
 news
 join
 supporters
 King
 event
 Iowa
 time
 16
 New
 2016
 did


In [66]:
sentences = []
for i in range(X_windows1252.sentence.count()):
    sentences.append(X_windows1252.sentence[i])
    
# now we try clustering
vectorizer2 = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=True,analyzer='word',ngram_range=(1, 2))

X2 = vectorizer2.fit_transform(sentences)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model3 = model.fit_transform(X2)

order_descent2= model.cluster_centers_.argsort()[:, ::-1]
terms2 = vectorizer2.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent2[i, :55]:
        print(' %s' % terms2[ind]),

Cluster 0:
 right
 just
 hilarious
 like
 dingo
 confused
 seen
 year
 haven
 try
 great
 want
 time
 employees
 don
 ve
 right entertainer
 entertainer
 haven seen
 post
 markets
 try hilarious
 make
 know
 really
 geotus
 guy
 lol
 lol hilarious
 american
 come
 dc
 people
 black
 white
 sen
 taking
 follow
 new
 alabama
 knew
 cut
 fun
 times
 happened
 life
 election
 mandate
 bonus
 mean
 movie
 happy
 obamacare
 just want
 tweet
Cluster 1:
 trump
 real
 deadfeddeedb
 moore
 coverage
 democrats
 wasn real
 wasn
 real referendum
 referendum trump
 referendum
 weeks
 think
 negative
 inbred
 blah
 money
 tax
 people
 working
 committee
 simultaneously
 vote
 jr
 intelligence
 great
 doesn
 president
 way
 broken long
 matter laws
 long trump
 laws broken
 doesn matter
 trump stopped
 trump jr
 stopped
 maga
 laws
 ass
 curse effect
 wait trump
 smug globalist
 smug
 globalist
 effect smug
 trump curse
 globalist ass
 wait
 matter
 bs
 effect
 broken
 fucking
 curse


In [67]:
sentences = []
for i in range(X1_windows1252.sentence.count()):
    sentences.append(X1_windows1252.sentence[i])
    
# now we try clustering
vectorizer3 = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=True,analyzer='word',ngram_range=(1, 2))

X3 = vectorizer3.fit_transform(sentences)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model4 = model.fit_transform(X3)

order_descent3= model.cluster_centers_.argsort()[:, ::-1]
terms3 = vectorizer3.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent3[i, :55]:
        print(' %s' % terms3[ind]),

Cluster 0:
 bernie
 sanders
 people
 like
 gt
 know
 bernie sanders
 campaign
 hillary
 want
 think
 just
 make
 vote
 support
 time
 issues
 bernies
 dont
 state
 work
 money
 candidate
 new
 need
 does
 help
 women
 clinton
 primary
 right
 political
 event
 election
 super
 party
 im
 voters
 important
 good
 really
 free
 states
 facebook
 way
 voting
 candidates
 news
 feel
 events
 post
 today
 information
 believe
 better
Cluster 1:
 sanders
 clinton
 bernie
 percent
 bernie sanders
 mentions
 just
 poll
 sanders clinton
 socialism
 democratic
 clinton sanders
 think
 article
 blackout
 new
 people
 11
 voters
 change clinton
 mentions change
 socialist
 campaign
 times
 hillary
 change
 good
 candidate
 edit
 bernies
 10
 animation
 support
 presidential
 money
 gt
 mr
 30
 right
 feedback
 like
 nh
 time
 12
 media
 gun
 21
 message
 day
 minutes
 help
 americans
 july
 america
 say


In [68]:
sentences = []
for i in range(X_windows1252.sentence.count()):
    sentences.append(X_windows1252.sentence[i])
    
# now we try clustering
vectorizer4 = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=True,analyzer='word',ngram_range=(1, 1))

X4 = vectorizer4.fit_transform(sentences)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model4 = model.fit_transform(X4)

order_descent4= model.cluster_centers_.argsort()[:, ::-1]
terms4 = vectorizer4.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent4[i, :55]:
        print(' %s' % terms4[ind]),

Cluster 0:
 real
 twitter
 referendum
 wasn
 inbred
 weeks
 trump
 tax
 times
 old
 working
 bs
 james
 taught
 soul
 wise
 christian
 vinnie
 scientologist
 pretending
 missing
 follow
 opportunity
 climate
 clinton
 change
 year
 article
 episode
 shameless
 redneck
 hillbilly
 watch
 don
 hard
 barsoomian
 lisa
 hat
 wearing
 getting
 shows
 family
 mr
 decreases
 college
 excited
 liability
 passed
 spent
 maga
 job
 thank
 life
 hoax
 fit
Cluster 1:
 trump
 just
 right
 like
 great
 seen
 year
 know
 people
 haven
 time
 make
 american
 come
 confused
 don
 really
 moore
 want
 dingo
 think
 post
 ve
 alabama
 matter
 guy
 reddit
 coverage
 geotus
 entertainer
 christmas
 markets
 today
 employees
 good
 ll
 america
 black
 white
 knew
 way
 seat
 election
 doesn
 taking
 feel
 happened
 did
 sad
 deadfeddeedb
 new
 vote
 cut
 life
 fun
Cluster 2:
 hilarious
 lol
 try
 zero
 fifth
 files
 file
 fil
 figures
 fight
 fiction
 finally
 ferris
 felt
 fellow
 feels
 feel
 final
 financ

In [70]:
sentences = []
for i in range(X1_windows1252.sentence.count()):
    sentences.append(X1_windows1252.sentence[i])
    
# now we try clustering
vectorizer5 = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=True,analyzer='word',ngram_range=(1, 1))

X5 = vectorizer5.fit_transform(sentences)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model5 = model.fit_transform(X5)

order_descent5= model.cluster_centers_.argsort()[:, ::-1]
terms5= vectorizer5.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent5[i, :55]:
        print(' %s' % terms5[ind]),

Cluster 0:
 bernie
 sanders
 people
 like
 just
 gt
 think
 know
 hillary
 support
 vote
 campaign
 candidate
 clinton
 time
 voters
 make
 issues
 want
 right
 state
 article
 new
 facebook
 dont
 party
 socialist
 political
 money
 good
 news
 percent
 event
 does
 work
 im
 help
 democratic
 important
 bernies
 need
 say
 women
 primary
 believe
 candidates
 page
 election
 post
 today
 voting
 edit
 actually
 issue
 did
Cluster 1:
 clinton
 mentions
 sanders
 blackout
 poll
 media
 16
 change
 10
 nh
 day
 22
 ugly
 12
 outlets
 11
 month
 2191443015477768
 17
 hillary
 biden
 good
 21
 eye
 events
 13
 18
 14
 bloomberg
 bernie
 data
 15
 nv
 wapo
 20
 30
 bad
 27
 new
 19
 saturday
 24
 23
 32
 excited
 sc
 ia
 28
 chase
 aft
 major
 rcp
 courtesy
 ghostofchinaski
 feelthe
Cluster 2:
 super
 campaign
 pack
 join
 bernies
 contribution
 monthly
 class
 money
 build
 pacs
 climate
 bernie
 corporations
 deal
 raised
 need
 tasks
 workers
 new
 billionaire
 ideas
 early
 speech
 hel

In [71]:
bias_statements = pd.read_csv("bias_sentences.csv")

In [72]:
for i in range(bias_statements['sentence'].count()):
    if isinstance(bias_statements.loc[i]['sentence'],str):
        my_byte = str.encode(bias_statements.loc[i]['sentence'])
        print(chardet.detect(my_byte)['encoding'])
                          

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

...

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

In [73]:
X_ascii = []

for i in range(bias_statements['sentence'].count()):
    if isinstance(bias_statements.loc[i]['sentence'],str):
        X_ascii.append(bias_statements.loc[i]['sentence'])



In [75]:
# now we try clustering
vectorizer = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=False,analyzer='word',ngram_range=(1, 1))

X = vectorizer.fit_transform(X_ascii)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model1 = model.fit_transform(X)

order_descent= model.cluster_centers_.argsort()[:, ::-1]
terms= vectorizer.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent[i, :55]:
        print(' %s' % terms[ind]),

Cluster 0:
 abortion
 percent
 rights
 month
 rates
 rate
 anti
 Treasury
 The
 Court
 said
 issue
 basis
 women
 points
 right
 late
 Supreme
 Mr
 clinics
 woman
 decision
 offered
 law
 auction
 abortions
 term
 year
 state
 In
 fell
 rose
 groups
 But
 support
 Six
 Short
 case
 Federal
 Bush
 life
 Roe
 legal
 family
 issues
 opponents
 today
 group
 Catholic
 day
 Wade
 new
 point
 clinic
 weekly
Cluster 1:
 Senate
 The
 Mr
 said
 Committee
 Senator
 Republican
 President
 year
 State
 New
 today
 Democrat
 leader
 vote
 But
 In
 Democratic
 state
 Bush
 new
 chairman
 majority
 Congress
 Assembly
 Republicans
 Democrats
 congress
 tax
 committee
 years
 United
 York
 States
 He
 week
 million
 passed
 Administration
 Federal
 campaign
 Governor
 nomination
 billion
 members
 approved
 caucuses
 legislation
 time
 law
 party
 veto
 Reagan
 budget
 political
Cluster 2:
 House
 Senate
 The
 passed
 White
 approved
 vote
 measure
 Representatives
 conference
 President
 legislation
 

In [76]:
# now we try clustering
vectorizer = TfidfVectorizer(encoding='Windows-1252',decode_error='ignore'
                             ,stop_words='english',lowercase=False,analyzer='word',ngram_range=(1, 1))

X = vectorizer.fit_transform(X_ascii)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model1 = model.fit_transform(X)

order_descent= model.cluster_centers_.argsort()[:, ::-1]
terms= vectorizer.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent[i, :55]:
        print(' %s' % terms[ind]),

Cluster 0:
 Senate
 The
 Mr
 House
 said
 Committee
 year
 Republican
 President
 Senator
 today
 vote
 passed
 State
 percent
 month
 New
 Democrat
 But
 In
 approved
 Bush
 Democratic
 Congress
 leader
 committee
 new
 week
 Democrats
 tax
 state
 chairman
 majority
 Republicans
 billion
 years
 measure
 legislation
 Assembly
 Administration
 members
 United
 million
 States
 congress
 budget
 Federal
 expected
 veto
 York
 He
 rates
 campaign
 Treasury
 rate
Cluster 1:
 abortion
 rights
 anti
 Court
 said
 women
 issue
 The
 right
 Supreme
 Mr
 clinics
 woman
 decision
 law
 abortions
 state
 groups
 support
 In
 But
 Bush
 case
 life
 Roe
 legal
 Federal
 family
 opponents
 group
 Catholic
 Wade
 clinic
 issues
 today
 planning
 debate
 position
 counseling
 Republican
 cases
 pregnancy
 New
 like
 say
 political
 advocates
 years
 President
 public
 constitutional
 laws
 new
 ruling
 He


In [77]:
# now we try clustering
vectorizer = TfidfVectorizer(encoding='ascii',decode_error='ignore'
                             ,stop_words='english',lowercase=False,analyzer='word',ngram_range=(1, 1))

X = vectorizer.fit_transform(X_ascii)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model1 = model.fit_transform(X)

order_descent= model.cluster_centers_.argsort()[:, ::-1]
terms= vectorizer.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent[i, :55]:
        print(' %s' % terms[ind]),

Cluster 0:
 Senate
 The
 Mr
 House
 said
 Committee
 year
 Republican
 President
 Senator
 today
 vote
 passed
 State
 percent
 month
 New
 Democrat
 But
 In
 approved
 Bush
 Democratic
 Congress
 leader
 committee
 new
 week
 Democrats
 tax
 state
 chairman
 majority
 Republicans
 billion
 years
 measure
 legislation
 Assembly
 Administration
 members
 United
 million
 States
 congress
 budget
 Federal
 expected
 veto
 York
 He
 rates
 campaign
 Treasury
 rate
Cluster 1:
 abortion
 rights
 anti
 Court
 said
 women
 issue
 The
 right
 Supreme
 Mr
 clinics
 woman
 decision
 law
 abortions
 state
 groups
 support
 In
 But
 Bush
 case
 life
 Roe
 legal
 Federal
 family
 opponents
 group
 Catholic
 Wade
 clinic
 issues
 today
 planning
 debate
 position
 counseling
 Republican
 cases
 pregnancy
 New
 like
 say
 political
 advocates
 years
 President
 public
 constitutional
 laws
 new
 ruling
 He


In [96]:
ibc_sentences = pd.read_csv("ibcData.csv", delimiter ='\t',names = ['sentence','label'],skiprows = 0)

In [97]:
ibc_sentences.head()

,sentence,label
0,Forcing middle-class workers to bear a greater...,liberal
1,Because it would not be worthwhile to bring a ...,liberal
2,"Indeed , Lind argues that high profits and hig...",liberal
3,"In fairness , it should be noted that he devot...",liberal
4,Psychological tactics are social control techn...,liberal


In [98]:
for i in range(ibc_sentences['sentence'].count()):
    if isinstance(ibc_sentences.loc[i]['sentence'],str):
        my_byte = str.encode(ibc_sentences.loc[i]['sentence'])
        print(chardet.detect(my_byte)['encoding'])

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
asci

ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii
ascii


In [99]:
X_ascii = []

for i in range(bias_statements['sentence'].count()):
    if isinstance(bias_statements.loc[i]['sentence'],str):
        X_ascii.append(bias_statements.loc[i]['sentence'])

In [100]:
for i in range(ibc_sentences['sentence'].count()):
    if isinstance(ibc_sentences.loc[i]['sentence'],str):
        X_ascii.append(ibc_sentences.loc[i]['sentence'])

In [102]:
# now we try clustering
vectorizer = TfidfVectorizer(encoding='ascii',decode_error='ignore'
                             ,stop_words='english',lowercase=True,analyzer='word',ngram_range=(1, 1))

X = vectorizer.fit_transform(X_ascii)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model1 = model.fit_transform(X)

order_descent= model.cluster_centers_.argsort()[:, ::-1]
terms= vectorizer.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent[i, :55]:
        print(' %s' % terms[ind]),

Cluster 0:
 senate
 mr
 said
 abortion
 house
 committee
 state
 new
 president
 republican
 year
 senator
 congress
 today
 vote
 passed
 bush
 rights
 states
 democratic
 democrat
 approved
 leader
 week
 tax
 democrats
 years
 majority
 republicans
 court
 legislation
 chairman
 law
 administration
 federal
 measure
 united
 party
 budget
 members
 billion
 assembly
 york
 support
 issue
 veto
 campaign
 government
 million
 expected
 anti
 public
 time
 political
 governor
Cluster 1:
 percent
 month
 rates
 rate
 treasury
 basis
 points
 late
 offered
 fell
 rose
 short
 term
 auction
 average
 point
 bills
 discount
 outstanding
 friday
 slightly
 weekly
 bid
 year
 mixed
 unchanged
 day
 dipped
 yield
 lowest
 market
 yesterday
 percentage
 week
 hundredths
 regular
 higher
 sold
 narrowly
 changed
 moved
 monday
 similar
 sharply
 little
 91
 new
 lower
 secondary
 issue
 eased
 90
 14
 13
 trading


In [103]:
# now we try clustering
vectorizer = TfidfVectorizer(encoding='ascii',decode_error='ignore'
                             ,stop_words='english',lowercase=True,analyzer='word',ngram_range=(1, 1))

X = vectorizer.fit_transform(X_ascii)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', random_state=42)

model1 = model.fit_transform(X)

order_descent= model.cluster_centers_.argsort()[:, ::-1]
terms= vectorizer.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_descent[i, :55]:
        print(' %s' % terms[ind]),

Cluster 0:
 abortion
 said
 mr
 congress
 new
 rights
 state
 year
 president
 anti
 law
 court
 party
 government
 bush
 women
 federal
 caucuses
 senate
 states
 issue
 national
 years
 tax
 right
 people
 today
 support
 million
 health
 public
 city
 make
 000
 caucus
 york
 political
 passed
 percent
 supreme
 veto
 like
 groups
 time
 money
 legislature
 republican
 say
 billion
 decision
 civil
 american
 life
 administration
 vote
Cluster 1:
 percent
 month
 rates
 rate
 basis
 treasury
 points
 late
 offered
 fell
 rose
 short
 term
 auction
 average
 point
 bills
 discount
 outstanding
 friday
 slightly
 weekly
 bid
 year
 mixed
 unchanged
 day
 dipped
 yield
 lowest
 market
 yesterday
 percentage
 week
 hundredths
 regular
 higher
 sold
 narrowly
 moved
 changed
 monday
 similar
 sharply
 little
 new
 lower
 91
 secondary
 issue
 eased
 14
 13
 90
 75
Cluster 2:
 senate
 house
 committee
 mr
 said
 republican
 senator
 state
 vote
 today
 president
 leader
 democrat
 new
 ap

In [104]:
cluster1 = []

for ind in order_descent[0, :55]:
    cluster1.append(terms[ind])

In [105]:
cluster2 = []

for ind in order_descent[2, :55]:
    cluster2.append(terms[ind])

In [107]:
cluster2

['senate',
 'house',
 'committee',
 'mr',
 'said',
 'republican',
 'senator',
 'state',
 'vote',
 'today',
 'president',
 'leader',
 'democrat',
 'new',
 'approved',
 'majority',
 'year',
 'chairman',
 'passed',
 'republicans',
 'democratic',
 'democrats',
 'week',
 'budget',
 'measure',
 'assembly',
 'legislation',
 'bush',
 'members',
 'states',
 'administration',
 'nomination',
 'united',
 'expected',
 'tax',
 'floor',
 'billion',
 'white',
 'campaign',
 'years',
 'hearings',
 'judiciary',
 'plan',
 'voted',
 'leaders',
 'finance',
 'judge',
 'banking',
 'reagan',
 'governor',
 'conference',
 'intelligence',
 'dole',
 'veto',
 'york']

In [108]:
cluster1

['abortion',
 'said',
 'mr',
 'congress',
 'new',
 'rights',
 'state',
 'year',
 'president',
 'anti',
 'law',
 'court',
 'party',
 'government',
 'bush',
 'women',
 'federal',
 'caucuses',
 'senate',
 'states',
 'issue',
 'national',
 'years',
 'tax',
 'right',
 'people',
 'today',
 'support',
 'million',
 'health',
 'public',
 'city',
 'make',
 '000',
 'caucus',
 'york',
 'political',
 'passed',
 'percent',
 'supreme',
 'veto',
 'like',
 'groups',
 'time',
 'money',
 'legislature',
 'republican',
 'say',
 'billion',
 'decision',
 'civil',
 'american',
 'life',
 'administration',
 'vote']

In [111]:
c3 = [c for c in cluster2 if c in cluster1]

In [112]:
c3

['senate',
 'mr',
 'said',
 'republican',
 'state',
 'vote',
 'today',
 'president',
 'new',
 'year',
 'passed',
 'bush',
 'states',
 'administration',
 'tax',
 'billion',
 'years',
 'veto',
 'york']

In [124]:
c1 = [c for c in cluster1 if c not in c3]

In [125]:
c2 = [c for c in cluster2 if c not in c3]

In [126]:
c1

['abortion',
 'congress',
 'rights',
 'anti',
 'law',
 'court',
 'party',
 'government',
 'women',
 'federal',
 'caucuses',
 'issue',
 'national',
 'right',
 'people',
 'support',
 'million',
 'health',
 'public',
 'city',
 'make',
 '000',
 'caucus',
 'political',
 'percent',
 'supreme',
 'like',
 'groups',
 'time',
 'money',
 'legislature',
 'say',
 'decision',
 'civil',
 'american',
 'life']

In [127]:
c2

['house',
 'committee',
 'senator',
 'leader',
 'democrat',
 'approved',
 'majority',
 'chairman',
 'republicans',
 'democratic',
 'democrats',
 'week',
 'budget',
 'measure',
 'assembly',
 'legislation',
 'members',
 'nomination',
 'united',
 'expected',
 'floor',
 'white',
 'campaign',
 'hearings',
 'judiciary',
 'plan',
 'voted',
 'leaders',
 'finance',
 'judge',
 'banking',
 'reagan',
 'governor',
 'conference',
 'intelligence',
 'dole']

In [117]:
c3_cluster1_ind = [i for i,c in enumerate(cluster1) if c in c3]

In [118]:
c3_cluster1_ind

[1, 2, 4, 6, 7, 8, 14, 18, 19, 22, 23, 26, 35, 37, 40, 46, 48, 53, 54]

In [119]:
c3_cluster2_ind = [i for i,c in enumerate(cluster2) if c in c3]

In [120]:
c3_cluster2_ind

[0, 3, 4, 5, 7, 8, 9, 10, 13, 16, 18, 27, 29, 30, 34, 36, 39, 53, 54]

In [128]:
for i in range(len(c3)):
    if c3_cluster1_ind[i] < c3_cluster2_ind[i]:
        c1.append(c3[i])
    elif c3_cluster1_ind[i] > c3_cluster2_ind[i]:
        c2.append(c3[i])
    else:
        continue

In [129]:
c1

['abortion',
 'congress',
 'rights',
 'anti',
 'law',
 'court',
 'party',
 'government',
 'women',
 'federal',
 'caucuses',
 'issue',
 'national',
 'right',
 'people',
 'support',
 'million',
 'health',
 'public',
 'city',
 'make',
 '000',
 'caucus',
 'political',
 'percent',
 'supreme',
 'like',
 'groups',
 'time',
 'money',
 'legislature',
 'say',
 'decision',
 'civil',
 'american',
 'life',
 'mr',
 'bush']

In [130]:
c2

['house',
 'committee',
 'senator',
 'leader',
 'democrat',
 'approved',
 'majority',
 'chairman',
 'republicans',
 'democratic',
 'democrats',
 'week',
 'budget',
 'measure',
 'assembly',
 'legislation',
 'members',
 'nomination',
 'united',
 'expected',
 'floor',
 'white',
 'campaign',
 'hearings',
 'judiciary',
 'plan',
 'voted',
 'leaders',
 'finance',
 'judge',
 'banking',
 'reagan',
 'governor',
 'conference',
 'intelligence',
 'dole',
 'senate',
 'republican',
 'today',
 'president',
 'new',
 'year',
 'passed',
 'states',
 'administration',
 'tax',
 'billion',
 'years']